In [0]:
from pyspark.sql.functions import *

#### Convert Rows to Column

<span style="color:magenta">**_used Pivot function_**</span>

In [0]:
data = [('Delhi','BMW',5500),('Mumbai','BMW',6500),('Kolkata','BMW',8500),
        ('Delhi','Audi',8000),('Mumbai','Audi',9500),('Kolkata','Audi',7500)]

schema = ['city','Cars','Price']

df = spark.createDataFrame(data, schema)
# Pivotinng the data
pivot_df = df.withColumn('price',col('price').cast('Integer')) \
        .groupBy('cars').pivot('city').agg(sum('Price'))
        
pivot_df.show()

+----+-----+-------+------+
|cars|Delhi|Kolkata|Mumbai|
+----+-----+-------+------+
|Audi| 8000|   7500|  9500|
| BMW| 5500|   8500|  6500|
+----+-----+-------+------+



#### How to keep any specific records always on Top 

In [0]:
data = [('UK','Mango',500),('India','apple',800),('US','apple',1200),('UK','Mango',2500),('India','apple',800),('US','apple',2200),('UK','apple',1500),('Russia','apple',1800),('US','apple',1200),('Aus','apple',1500),('India','Mango',500),('US','Mango',1100)]

schema = ['country','Product','Sales']

df = spark.createDataFrame(data, schema)
df.show(5)

+-------+-------+-----+
|country|Product|Sales|
+-------+-------+-----+
|     UK|  Mango|  500|
|  India|  apple|  800|
|     US|  apple| 1200|
|     UK|  Mango| 2500|
|  India|  apple|  800|
+-------+-------+-----+
only showing top 5 rows



In [0]:
# Here Total_sales of India is less than US and UK but India is showing on Top position after ordering

df_top =df.withColumn('sales',col('sales').cast('Integer'))\
        .groupBy('country').agg(sum('sales').alias('Total_sales'))\
        .withColumn('rank',when(col('country')=='India',1).otherwise(0)).sort('rank','Total_sales',ascending=[False,False]).drop('rank')
df_top.show()

+-------+-----------+
|country|Total_sales|
+-------+-----------+
|  India|       2100|
|     US|       5700|
|     UK|       4500|
| Russia|       1800|
|    Aus|       1500|
+-------+-----------+



In [0]:
#  Sql method
df.createOrReplaceTempView('Sales')

In [0]:
%sql
select Country,sum(sales) as Total_Sales 
from sales
group by Country
order by 
      (case when Country = 'India' then 1 else 0 end)  desc ,Total_sales desc;

Country,Total_Sales
India,2100
US,5700
UK,4500
Russia,1800
Aus,1500
